In [ ]:
import requests
from bs4 import BeautifulSoup



In [ ]:
# Last name
last_name = "son"  # Replace with the desired last name

# URL with placeholder for last name
URL = 'https://www.bbc.com/search?q=premier%20league%20{}'

# Replace the placeholder with the last name
URL = URL.format(last_name)

# Send a GET request to the URL
response = requests.get(URL)

# Print the response
print(response)

<Response [200]>


In [ ]:
soup = BeautifulSoup(response.content,'html.parser')
#print(soup)

In [ ]:
headlines = soup.find_all(class_='sc-4fedabc7-3 bvDsJq')

# Find all card descriptions
descriptions = soup.find_all(class_='sc-ae29827d-0 cNPpME')

# Print headlines and descriptions
for headline, description in zip(headlines, descriptions):
    print("Headline:", headline.text)
    print("Description:", description.text)

Headline: Postecoglou praises 'brilliant' Son after Spurs win
Description: Tottenham boss Ange Postecoglou was full of praise for Son Heung-min after his goal secured a 2-1 Premier League win over Luton.
Headline: Planning for double gameweeks - fantasy football tips
Description: It's time for Fantasy Football players to start planning the rest of their season - Alistair Bruce-Ball previews gameweek 30.
Headline: Palmer a tempting captain - fantasy football tips
Description: Cole Palmer is an obvious candidate for the captaincy, but there is plenty of competition. says Alistair Bruce-Ball.
Headline: Little chance of late entrants to Scotland squad - gossip
Description: Latest from Scotland camp, Rangers player's father on future and Aberdeen captain on Crohn's Disease battle.
Headline: Heading damaged my body and I played on autopilot - Varane
Description: Raphael Varane says he has "damaged his body" because of the continued impact of heading the ball.
Headline: Who is a warrior? - Ga

In [ ]:
from textblob import TextBlob



In [ ]:
# Loop through headlines and descriptions
for headline, description in zip(headlines, descriptions):
    # Combine headline and description text
    text = headline.text + ' ' + description.text

    # Perform sentiment analysis
    analysis = TextBlob(text)

    # Print sentiment polarity and subjectivity
    print("Text:", text)
    print("Sentiment Polarity:", analysis.sentiment.polarity)
    print("Sentiment Subjectivity:", analysis.sentiment.subjectivity)
    print("\n")

Text: Postecoglou praises 'brilliant' Son after Spurs win Tottenham boss Ange Postecoglou was full of praise for Son Heung-min after his goal secured a 2-1 Premier League win over Luton.
Sentiment Polarity: 0.7125000000000001
Sentiment Subjectivity: 0.5875


Text: Planning for double gameweeks - fantasy football tips It's time for Fantasy Football players to start planning the rest of their season - Alistair Bruce-Ball previews gameweek 30.
Sentiment Polarity: 0.0
Sentiment Subjectivity: 0.0


Text: Palmer a tempting captain - fantasy football tips Cole Palmer is an obvious candidate for the captaincy, but there is plenty of competition. says Alistair Bruce-Ball.
Sentiment Polarity: 0.0
Sentiment Subjectivity: 0.5


Text: Little chance of late entrants to Scotland squad - gossip Latest from Scotland camp, Rangers player's father on future and Aberdeen captain on Crohn's Disease battle.
Sentiment Polarity: 0.0031250000000000028
Sentiment Subjectivity: 0.53125


Text: Heading damaged my 

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

# Initialize lists to store sentiment polarities
sentiment_polarities = []

# Perform sentiment analysis for each data point
for headline, description in zip(headlines, descriptions):
    # Combine headline and description
    text = headline.text + ' ' + description.text

    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)

    # Perform inference
    outputs = model(**inputs)

    # Extract predicted sentiment
    predicted_class = torch.argmax(outputs.logits).item()

    # Map predicted class to sentiment label
    sentiments = ['negative', 'neutral', 'positive']
    predicted_sentiment = sentiments[predicted_class]

    # Print results
    #print("Headline:", headline)
    #print("Description:", description)
    print("text:", text)
    print("Predicted Sentiment:", predicted_sentiment)
    print()

    # Calculate sentiment polarity
    if predicted_sentiment == 'positive':
        sentiment_polarity = 1
    elif predicted_sentiment == 'negative':
        sentiment_polarity = -1
    else:
        sentiment_polarity = 0

    # Append sentiment polarity to the list
    sentiment_polarities.append(sentiment_polarity)

# Calculate aggregated sentiment polarity
aggregated_sentiment = sum(sentiment_polarities) / len(sentiment_polarities)
print("Aggregated Sentiment Polarity:", aggregated_sentiment)

text: Postecoglou praises 'brilliant' Son after Spurs win Tottenham boss Ange Postecoglou was full of praise for Son Heung-min after his goal secured a 2-1 Premier League win over Luton.
Predicted Sentiment: neutral

text: Planning for double gameweeks - fantasy football tips It's time for Fantasy Football players to start planning the rest of their season - Alistair Bruce-Ball previews gameweek 30.
Predicted Sentiment: neutral

text: Palmer a tempting captain - fantasy football tips Cole Palmer is an obvious candidate for the captaincy, but there is plenty of competition. says Alistair Bruce-Ball.
Predicted Sentiment: negative

text: Little chance of late entrants to Scotland squad - gossip Latest from Scotland camp, Rangers player's father on future and Aberdeen captain on Crohn's Disease battle.
Predicted Sentiment: neutral

text: Heading damaged my body and I played on autopilot - Varane Raphael Varane says he has "damaged his body" because of the continued impact of heading the ba

In [ ]:
import csv
with open('sentiment_data.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Aggregated Sentiment'])


In [ ]:
with open('sentiment_data.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([last_name, aggregated_sentiment])

print("Data has been written to sentiment_data.csv")

Data has been written to sentiment_data.csv
